<a href="https://colab.research.google.com/github/AnastasiaZAYU/elliptic-curve-crypto-library/blob/main/notebooks/02_crypto_protocols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Комп'ютерний практикум #2: Реалізація класичних криптосистем на базі еліптичних кривих

In [1]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00


In [99]:
import random
import hashlib
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

In [ ]:
import sys
import os
if 'google.colab' in str(get_ipython()):
    if not os.path.exists('elliptic-curve-crypto-library'):
        !git clone https://github.com/AnastasiaZAYU/elliptic-curve-crypto-library.git
    %cd elliptic-curve-crypto-library/notebooks
    
sys.path.append(os.path.abspath('../'))

from src import EllipticCurve, CurvePoint

In [101]:
#параметри еліптичної кривої P-224
p = '0xffffffffffffffffffffffffffffffff000000000000000000000001'
a = '0xfffffffffffffffffffffffffffffffefffffffffffffffffffffffe'
b = '0xb4050a850c04b3abf54132565044b0b7d7bfd8ba270b39432355ffb4'
n = '0xffffffffffffffffffffffffffff16a2e0b8f03e13dd29455c5c2a3d'

curve = EllipticCurve(p, a, b, n)
P = CurvePoint(curve, '0xb70e0cbd6bb4bf7f321390b94a03c1d356c21122343280d6115c1d21', '0xbd376388b5f723fb4c22dfe6cd4375a05a07476444d5819985007e34', '0x1')

### 1. Реалізувати схему ключового обміну Діффі-Геллмана на еліптичних кривих та перевірити коретність узгодження спільного секрету.

In [102]:
#генерація відкритого та приватного ключів
def generate_keys(P):
  d = random.randint(2, int(n, 16))
  Q = P.PointMultiply(d)
  return d, Q

In [103]:
da, Qa = generate_keys(P) #ключі Аліси
db, Qb = generate_keys(P) #ключі Боба

In [104]:
Sb = Qa.PointMultiply(db) #спільний секрет, який обчислив Боб

In [105]:
Sa = Qb.PointMultiply(da) #спільний секрет, який обчислила Аліса

In [106]:
#перевірка коретності узгодження спільного секрету
print(Sb.ComparePoints(Sa))

True


### 2. Реалізувати алгоритм направленого шифрування на базі ефемерного ключового обміну Діффі-Геллмана на еліптичних кривих та перевірити коректність, зашифрувавши довільне повідомлення.

In [107]:
#функція шифрування повідомлення
def encrypt(M, P, Qb):
  k = get_random_bytes(32)
  Enc = AES.new(k, AES.MODE_CBC)
  Cm = Enc.iv + Enc.encrypt(pad(M, AES.block_size))

  ea, Qa = generate_keys(P)
  S = Qb.PointMultiply(ea).ToAffine()
  Sx = S.X.to_bytes(32, byteorder='big')

  Wrap = AES.new(Sx, AES.MODE_CBC)
  Ck = Wrap.iv + Wrap.encrypt(pad(k, AES.block_size))

  return (Qa, Ck, Cm)

In [108]:
#функція розшифрування повідомлення
def decrypt(Env, db):
  S = Env[0].PointMultiply(db).ToAffine()
  Sx = S.X.to_bytes(32, byteorder='big')

  Unwrap = AES.new(Sx, AES.MODE_CBC, Env[1][:16])
  k = unpad(Unwrap.decrypt(Env[1][16:]), AES.block_size)

  Dec = AES.new(k, AES.MODE_CBC, Env[2][:16])
  M = unpad(Dec.decrypt(Env[2][16:]), AES.block_size)

  return M

In [109]:
db, Qb = generate_keys(P) #ключі Боба

In [110]:
M = b"The striped hyena is a primarily nocturnal animal, which typically only leaves its den at the onset of total darkness, returning before sunrise. Striped hyenas typically live alone or in pairs, though groups of up to seven animals are known in Libya. They are generally not territorial animals, with home ranges of different groups often overlapping each other."
Env = encrypt(M, P, Qb) #зашифрований конверт

In [111]:
Mb = decrypt(Env, db) #повідомлення, яке розшифрував Боб

In [112]:
#перевірка коректності шифрування та розшифрування повідомлення
print(M == Mb)

True


### 3. Реалізувати схему цифрового підпису згідно алгоритму ECDSA та перевірити коректність.

In [113]:
#геш-функція SHA-3 256
def H(M, mod):
  sha = hashlib.sha3_256()
  sha.update(M)
  return int(sha.hexdigest(), 16) % mod

In [114]:
#підписання повідомлення
def sign(M, P, da):
  mod = int(n, 16)
  h = H(M, mod)

  r = 0
  while r == 0:
    k = random.randint(2, mod)
    point = P.PointMultiply(k).ToAffine()
    r = point.X % mod
  s = (pow(k, -1, mod) * (h + da * r)) % mod

  return r, s

In [115]:
#перевірка підпису повідомлення
def verify(M, r, s, Qa):
  mod = int(n, 16)
  h = H(M, mod)

  u1 = (pow(s, -1, mod) * h) % mod
  u2 = (pow(s, -1, mod) * r) % mod

  point = P.PointMultiply(u1).PointAdd(Qa.PointMultiply(u2)).ToAffine()
  v = point.X % mod

  return v == r

In [116]:
da, Qa = generate_keys(P) #ключі Аліси

In [117]:
M = b"The Machinist is a 2004 psychological thriller film directed by Brad Anderson and written by Scott Kosar. It stars Christian Bale as the title character, a machinist struggling with paranoia and delusion after being unable to sleep for an entire year."
r, s = sign(M, P, da) #підпис повідомлення

In [118]:
##перевірка коректності підписання повідомлення
print(verify(M, r, s, Qa))

True
